# <span style="color:darkblue"> Lecture 13 - Replacing and recoding variables</span>

<font size = "5">

- Starting a new module on manipulating data
- We will discuss NaNs and how to clean data


# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">
Key libraries

In [3]:
import numpy as np
import pandas as pd

<font size = "5">

Read dataset on car racing circuits

- https://en.wikipedia.org/wiki/Formula_One <br>
- [See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [4]:
circuits = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

The dataset "codebook" is a table with ...

- Key column information
- Main things:  Field, Type, and Description

<img src="figures/codebook_circuits.png" alt="drawing" width="500"/>


In [32]:
# The codebook contains basic about the columns
# "Field" is the name given to the name of the column
# "Type"  is the variable type (with the number of characters in parenthesis):
#         integer (int)
#         string (varchar - "variable character") 
#         float (float)
# "Description" contains a label with the content of the variable

<font size = "5">

Quick Discussion:
- What does the "alt" column represent?
- What does "varchar(255)" mean?

# <span style="color:darkblue"> II. NaNs </span>

<font size = "5">

- Means "Not a Number"
- Used to denote missing values

In [6]:
# "NaNs" are a special number, available in the "np" library

np.nan

nan

<font size = "5">

Operations on arrays with NaNs

In [8]:
# Create two array with and without "NaNs"
# The "np.array()" functions converts a list to an array

vec_without_nans = np.array(   [1,1,1]   )
vec_with_nans = np.array(  [np.nan,4,5] )

# When you add the vectors
# you will produce an error on any entries with "NaNs"
print(vec_without_nans * vec_with_nans)
print(vec_without_nans / vec_with_nans)
print(vec_without_nans + vec_with_nans)
print(vec_without_nans - vec_with_nans)

#Any operation with a "NaNs" will produce a "NaNs"

[nan  4.  5.]
[ nan 0.25 0.2 ]
[nan  5.  6.]
[nan -3. -4.]


<font size = "5">

Summary statistics with NaNs

In [9]:
# Some summary statistics will not work if there are "NaNs"
# The "np.mean()" doesn't work if the mean includes "NaNs"
print(np.mean(vec_with_nans))

# Some commands ignore the "nans"
# The "np.nanmean()" computes the mean over the numeric obvservations
print(np.nanmean(vec_with_nans))

nan
4.5


<font size = "5">

Pandas summary statistics ignore NaNs

In [10]:
# This command creates an empty dataframe
# then creates a new column called "x" and computes its mean
# Note: If a column contains missing values, then the "mean" won't be 
#       representative of the whole sample

dataset = pd.DataFrame([])
dataset["x"] = vec_with_nans
dataset["x"].mean()

#Deafult for summary statistics is pandas is to ignore "NaNs"

4.5

# <span style="color:darkblue"> II. Data Cleaning</span>

<font size = "5">

- Data collection isn't perfect!
- Need to adjust values with incorrect formatting

<font size = "5">
Get data types

In [11]:
# Produces a list with the data types of each column
# Columns that say "object" have either strings or 
# a mix of string and numeric values

# "." is called a pipe operator

circuits.dtypes

circuitId       int64
circuitRef     object
name           object
location       object
country        object
lat           float64
lng           float64
alt            object
url            object
dtype: object

<font size = "5">

Check rows with numeric values

In [12]:
# The ".str.isnumeric()" checks whether each row is numeric or not.
# Using the "." twice is an example of "piping"
# which refers to chaining two commands "str" and "isnumeric()"

circuits["alt"].str.isnumeric()

0      True
1      True
2      True
3      True
4      True
      ...  
72     True
73     True
74     True
75    False
76    False
Name: alt, Length: 77, dtype: bool

<font size = "5">

Extract list of non-numeric values

In [25]:
# We can reference subattributes of columns in ".query()"
# The pd.unique() function extracts unique values from a list

subset      = circuits.query("alt.str.isnumeric() == False")
list_unique = pd.unique(subset["alt"])
print(list_unique)


[]


<font size = "5">

Replace certain values

In [14]:
# "list_old" encodes values we want to change
# "list_new" encodes the values that will "replace" the old
list_old = ['\\N','-7']
list_new = [np.nan,-7]

# This command replaces the values of the "alt" column
circuits["alt"] = circuits["alt"].replace(list_old, list_new)

# Note: The option "inplace = True" permanently modifies the column
# circuits["alt"].replace(list_old, list_new, inplace = True)


<font size = "5">

Store a "cleaned" dataset

In [15]:
# After the cleaning process is done, you may want to store the dataset again
# It's recommended to do this in a separate file from the original
# That way you can always go back to the original if you made a coding error

circuits.to_csv("data_clean/circuits.csv")


<font size = "5">
Try it yourself!

- Use ".replace()" with the "country" column
- Replace "UK" with "United Kingdom"

In [16]:
# Write your own code

circuits["country"] = circuits["country"].replace("UK","United Kingdom")
circuits["country"] = circuits["country"].replace(["UK"],["United Kingdom"])
circuits.to_csv("data_clean/circuits.csv")



<font size = "5">
Try it yourself!

- What is the column type of "lat" or "lng"?
- Does it have any string variables?

In [17]:
# Write your own code

print(circuits["lat"].dtype)
print(circuits["lng"].dtype)





float64
float64


<font size = "5">

# <span style="color:darkblue"> II. Recoding Numeric Variables </span>



<font size = "5">

<span style="color:red"> Controlled Pitfall </span> Computing a mean for a non-numeric column

In [18]:
# Uncomment this command to see the error
# The following error occurs because the data type
# is not numeric
circuits["alt"].mean()

TypeError: can only concatenate str (not "int") to str

<font size = "5">

Convert column to numeric

In [19]:
# pd.to_numeric() converts a column to numeric
# Before you use this option, make sure to "clean" the variable
# as we did before by checking what the non-numeric values are

circuits["alt_numeric"] = pd.to_numeric(circuits["alt"])
print(circuits["alt_numeric"].mean())

248.1891891891892


<font size = "5">

Recode values based on an interval <br>

$ \qquad x_{bin} = \begin{cases} ``A" &\text{ if \quad} x_1 < x \le x_2 \\
                                  ``B" &\text{ if \quad} x_2 < x \le x_3 \end{cases} $



In [20]:
bins_x = [0,2500, 5000]
labels_x = ["Between 0 and 2500",
            "Between 2500 and 5000"]

circuits["bins_alt"] = pd.cut(circuits["alt_numeric"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)

# Note: if we set bins_x = [float("-inf"),2500, float("inf")]
#       then intervals are "Less than or equal to 2500" and "Above 2500"
#       float("inf") and float("-inf") represent infinity and negative infinity
#       The "right" command indicates that the right interval is
#       "less than or equal to"

<font size = "5">
Try it yourself!

- Create a new variable "hemisphere"
- Encode lattitude in (-90 and 0] as "south"
- Encode lattitude in (0 and 90] as "north"

In [21]:

# Write your own code

bins_y = [-90,0, 90]
labels_y = ["South",
            "North"]

circuits["hemisphere"] = pd.cut(circuits["lat"],
                              bins = bins_y,
                              right = True,
                              labels = labels_y)



